In [ ]:
#!pip install simple_salesforce
#!pip install pymysql
#!pip install sqlalchemy

In [1]:
import json
from sqlalchemy import create_engine
from datetime import date
import pandas as pd
import pymysql
pymysql.install_as_MySQLdb()

In [2]:
# Make sure to use your own `config.py` file. Consider ensuring that these variable names are in sync
from config import sf_username, sf_password, sf_security_token
from config import remote_db_endpoint, remote_db_port
from config import remote_db_name, remote_db_user, remote_db_pwd

In [59]:
from simple_salesforce import Salesforce
sf = Salesforce(username=sf_username, password=sf_password, security_token=sf_security_token)

In [60]:
engine = create_engine(f"mysql://{remote_db_user}:{remote_db_pwd}@{remote_db_endpoint}:{remote_db_port}/{remote_db_name}")
conn = engine.connect()

## Prepare ETL for the Course data

In [5]:
course_data_df = pd.read_sql("SELECT * FROM course", conn)
course_data_df.head(30)

,ID_Course,CourseCode,CourseName,CreditHours,BootCampCourse
0,1,BC-DATAVIZ,Data Visualization and Analytics,12,1
1,2,BC-WEBDEV,Full Stack Web Development,12,1
2,3,BC-UIUX,User Interface/User Experience,12,1
3,4,CIS-349,Introduction to Databases,5,0
4,5,CIS-405,Database Programming,5,0
5,6,CIS-438,Database Administration,5,0
6,7,CIS-430,Business Systems Programming I,5,0
7,8,CIS-435,Business Systems Programming II,5,0


In [6]:
course_data_df.rename(columns={
    'CourseCode':'Course_Code__c',
    'CourseName':'Course_Name__c',
    'CreditHours':'Credit_Hours__c',
    'BootCampCourse':'Boot_Camp_Course__c',    
}, inplace=True)
course_data_df.head()

,ID_Course,Course_Code__c,Course_Name__c,Credit_Hours__c,Boot_Camp_Course__c
0,1,BC-DATAVIZ,Data Visualization and Analytics,12,1
1,2,BC-WEBDEV,Full Stack Web Development,12,1
2,3,BC-UIUX,User Interface/User Experience,12,1
3,4,CIS-349,Introduction to Databases,5,0
4,5,CIS-405,Database Programming,5,0


In [7]:
course_data_df = course_data_df[['Course_Code__c', 'Course_Name__c', 'Credit_Hours__c', 'Boot_Camp_Course__c']]
course_data_df

,Course_Code__c,Course_Name__c,Credit_Hours__c,Boot_Camp_Course__c
0,BC-DATAVIZ,Data Visualization and Analytics,12,1
1,BC-WEBDEV,Full Stack Web Development,12,1
2,BC-UIUX,User Interface/User Experience,12,1
3,CIS-349,Introduction to Databases,5,0
4,CIS-405,Database Programming,5,0
5,CIS-438,Database Administration,5,0
6,CIS-430,Business Systems Programming I,5,0
7,CIS-435,Business Systems Programming II,5,0


In [8]:
course_data_records = course_data_df.to_dict('records')
course_data_records

[{'Course_Code__c': 'BC-DATAVIZ',
  'Course_Name__c': 'Data Visualization and Analytics',
  'Credit_Hours__c': 12,
  'Boot_Camp_Course__c': 1},
 {'Course_Code__c': 'BC-WEBDEV',
  'Course_Name__c': 'Full Stack Web Development',
  'Credit_Hours__c': 12,
  'Boot_Camp_Course__c': 1},
 {'Course_Code__c': 'BC-UIUX',
  'Course_Name__c': 'User Interface/User Experience',
  'Credit_Hours__c': 12,
  'Boot_Camp_Course__c': 1},
 {'Course_Code__c': 'CIS-349',
  'Course_Name__c': 'Introduction to Databases',
  'Credit_Hours__c': 5,
  'Boot_Camp_Course__c': 0},
 {'Course_Code__c': 'CIS-405',
  'Course_Name__c': 'Database Programming',
  'Credit_Hours__c': 5,
  'Boot_Camp_Course__c': 0},
 {'Course_Code__c': 'CIS-438',
  'Course_Name__c': 'Database Administration',
  'Credit_Hours__c': 5,
  'Boot_Camp_Course__c': 0},
 {'Course_Code__c': 'CIS-430',
  'Course_Name__c': 'Business Systems Programming I',
  'Credit_Hours__c': 5,
  'Boot_Camp_Course__c': 0},
 {'Course_Code__c': 'CIS-435',
  'Course_Name__c':

In [9]:
for rec in course_data_records:

    record = {
        'Course_Code__c': rec['Course_Code__c'],
        'Course_Name__c': rec['Course_Name__c'],
        'Credit_Hours__c': rec['Credit_Hours__c'],
        'Boot_Camp_Course__c': rec['Boot_Camp_Course__c'],
    }
    
    try:
        sf.Course__c.create(record)
    except Exception as e:
        print(e)

In [ ]:
# Bulk 
#sf.bulk.Course__c.insert(course_data_records)

## Create Course Lookup Table
You will use this later to crosswalk the course code with the primary key from the `Course` table

It is important to note that we will be querying **Salesforce** to retrieve the record IDs 

In [10]:
course_lookup_list = []

# The `Name` column in the primary key in Salesforce objects
# The Salesforce query language is called SOQL 
data = sf.query_all_iter("SELECT Course_Code__c, Name FROM Course__c")
for row in data:
    rec = {
        'ID_Course__c': row['Name'], # this is a critical line of code
        'Course_Code__c': row['Course_Code__c']
    }
    course_lookup_list.append(rec)
    
course_lookup_list

[{'ID_Course__c': 'a004x000003VR92', 'Course_Code__c': 'BC-DATAVIZ'},
 {'ID_Course__c': 'a004x000003VR97', 'Course_Code__c': 'BC-WEBDEV'},
 {'ID_Course__c': 'a004x000003VR9W', 'Course_Code__c': 'CIS-430'},
 {'ID_Course__c': 'a004x000003VR9C', 'Course_Code__c': 'BC-UIUX'},
 {'ID_Course__c': 'a004x000003VR9H', 'Course_Code__c': 'CIS-349'},
 {'ID_Course__c': 'a004x000003VR9M', 'Course_Code__c': 'CIS-405'},
 {'ID_Course__c': 'a004x000003VR9R', 'Course_Code__c': 'CIS-438'},
 {'ID_Course__c': 'a004x000003VR9b', 'Course_Code__c': 'CIS-435'}]

In [11]:
course_lookup_df = pd.DataFrame(course_lookup_list)
course_lookup_df

,ID_Course__c,Course_Code__c
0,a004x000003VR92,BC-DATAVIZ
1,a004x000003VR97,BC-WEBDEV
2,a004x000003VR9W,CIS-430
3,a004x000003VR9C,BC-UIUX
4,a004x000003VR9H,CIS-349
5,a004x000003VR9M,CIS-405
6,a004x000003VR9R,CIS-438
7,a004x000003VR9b,CIS-435


In [12]:
# Query the `Class` table from MySQL
query = '''
    SELECT 
        co.CourseCode,
        cl.*
    FROM 
        class cl
        INNER JOIN course co
        ON cl.ID_Course = co.ID_Course

'''

class_data_df = pd.read_sql(query, conn)
class_data_df.head()

,CourseCode,ID_Class,ID_Course,Section,StartDate,EndDate
0,BC-DATAVIZ,1,1,GWU-ARL-DATA-PT-09-0,2020-09-16,2020-03-14
1,BC-DATAVIZ,2,1,GWDC201805DATA3,2018-05-15,2018-11-08
2,BC-WEBDEV,3,2,GWARL201905WEB3,2019-05-14,2019-11-07
3,BC-UIUX,4,3,GWARL201905UIUX3,2019-05-14,2019-11-07


In [13]:
class_data_df.rename(columns={
    'CourseCode':'Course_Code__c',
    'Section':'Section__c',
    'StartDate':'Start_Date__c',
    'EndDate':'End_Date__c',
    'CourseCode':'Course_Code__c'
}, inplace=True)


class_data_df = class_data_df[['Course_Code__c', 'Section__c', 'Start_Date__c', 'End_Date__c']]
class_data_df

,Course_Code__c,Section__c,Start_Date__c,End_Date__c
0,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,2020-09-16,2020-03-14
1,BC-DATAVIZ,GWDC201805DATA3,2018-05-15,2018-11-08
2,BC-WEBDEV,GWARL201905WEB3,2019-05-14,2019-11-07
3,BC-UIUX,GWARL201905UIUX3,2019-05-14,2019-11-07


## Join the Class DataFrame with the Course lookup table
This join is necessary to successfully lookup the foreign key for the Course table 

In [14]:
class_data_df = pd.merge(class_data_df, course_lookup_df, how='left')
#class_data_df.drop(columns = ['ID_Class','ID_Course','CourseName','CreditHours','BootCampCourse','Course_Code__c'], inplace=True)

class_data_df.head()

,Course_Code__c,Section__c,Start_Date__c,End_Date__c,ID_Course__c
0,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,2020-09-16,2020-03-14,a004x000003VR92
1,BC-DATAVIZ,GWDC201805DATA3,2018-05-15,2018-11-08,a004x000003VR92
2,BC-WEBDEV,GWARL201905WEB3,2019-05-14,2019-11-07,a004x000003VR97
3,BC-UIUX,GWARL201905UIUX3,2019-05-14,2019-11-07,a004x000003VR9C


In [15]:
class_data_df
class_data_df['Start_Date__c'] = pd.to_datetime(class_data_df['Start_Date__c']).dt.date
class_data_df['End_Date__c'] = pd.to_datetime(class_data_df['End_Date__c']).dt.date

class_data_df.head()


,Course_Code__c,Section__c,Start_Date__c,End_Date__c,ID_Course__c
0,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,2020-09-16,2020-03-14,a004x000003VR92
1,BC-DATAVIZ,GWDC201805DATA3,2018-05-15,2018-11-08,a004x000003VR92
2,BC-WEBDEV,GWARL201905WEB3,2019-05-14,2019-11-07,a004x000003VR97
3,BC-UIUX,GWARL201905UIUX3,2019-05-14,2019-11-07,a004x000003VR9C


In [16]:
class_data_records = class_data_df.to_dict(orient='records')
class_data_records

[{'Course_Code__c': 'BC-DATAVIZ',
  'Section__c': 'GWU-ARL-DATA-PT-09-0',
  'Start_Date__c': datetime.date(2020, 9, 16),
  'End_Date__c': datetime.date(2020, 3, 14),
  'ID_Course__c': 'a004x000003VR92'},
 {'Course_Code__c': 'BC-DATAVIZ',
  'Section__c': 'GWDC201805DATA3',
  'Start_Date__c': datetime.date(2018, 5, 15),
  'End_Date__c': datetime.date(2018, 11, 8),
  'ID_Course__c': 'a004x000003VR92'},
 {'Course_Code__c': 'BC-WEBDEV',
  'Section__c': 'GWARL201905WEB3',
  'Start_Date__c': datetime.date(2019, 5, 14),
  'End_Date__c': datetime.date(2019, 11, 7),
  'ID_Course__c': 'a004x000003VR97'},
 {'Course_Code__c': 'BC-UIUX',
  'Section__c': 'GWARL201905UIUX3',
  'Start_Date__c': datetime.date(2019, 5, 14),
  'End_Date__c': datetime.date(2019, 11, 7),
  'ID_Course__c': 'a004x000003VR9C'}]

## Insert `Class` Records into Salesforce

In [17]:
for rec in class_data_records:
 
    record = {
        'ID_Course__c': rec['ID_Course__c'],
        'Section__c': rec['Section__c'],
        'Start_Date__c': str(rec['Start_Date__c']),
        'End_Date__c': str(rec['End_Date__c']),
    }
    
    try:
        sf.Class__C.create(record)
    except Exception as e:
        print(e)

## Insert Staff Data into Saleforce

In [18]:
#Querry MySQL and transorm data
staff_data_df = pd.read_sql("SELECT * FROM staff", conn)
staff_data_df

,ID_Staff,EmployeeID,LastName,FirstName,MiddleName,BirthDate
0,1,000184220,Luongo,Darick,Nico,1995-07-05
1,2,000130109,Sanford,Gemini,Blair,1992-04-22
2,3,000160655,Williams,Dartanion,De Angelo,1993-05-21
3,4,000159108,Rodney,Heather,Nicole,1994-06-07


In [19]:
staff_data_df.rename(columns={
    'EmployeeID':'EmployeeID__c',
    'LastName':'LastName__c',
    'FirstName':'FirstName__c',
    'MiddleName':'MiddleName__c', 
    'BirthDate' : 'BirthDate__c'
}, inplace=True)
staff_data_df.head()

,ID_Staff,EmployeeID__c,LastName__c,FirstName__c,MiddleName__c,BirthDate__c
0,1,000184220,Luongo,Darick,Nico,1995-07-05
1,2,000130109,Sanford,Gemini,Blair,1992-04-22
2,3,000160655,Williams,Dartanion,De Angelo,1993-05-21
3,4,000159108,Rodney,Heather,Nicole,1994-06-07


In [20]:
#Select columns to be exported to SF
staff_data_df.drop(columns = ['ID_Staff'], inplace=True)
staff_data_df['BirthDate__c'] = pd.to_datetime(staff_data_df['BirthDate__c']).dt.date
staff_data_df

,EmployeeID__c,LastName__c,FirstName__c,MiddleName__c,BirthDate__c
0,000184220,Luongo,Darick,Nico,1995-07-05
1,000130109,Sanford,Gemini,Blair,1992-04-22
2,000160655,Williams,Dartanion,De Angelo,1993-05-21
3,000159108,Rodney,Heather,Nicole,1994-06-07


In [21]:
#convert dataframe in a dictionary for SF 
staff_data_records = staff_data_df.to_dict('records')
staff_data_records


[{'EmployeeID__c': '000184220',
  'LastName__c': 'Luongo',
  'FirstName__c': 'Darick',
  'MiddleName__c': 'Nico',
  'BirthDate__c': datetime.date(1995, 7, 5)},
 {'EmployeeID__c': '000130109',
  'LastName__c': 'Sanford',
  'FirstName__c': 'Gemini',
  'MiddleName__c': 'Blair',
  'BirthDate__c': datetime.date(1992, 4, 22)},
 {'EmployeeID__c': '000160655',
  'LastName__c': 'Williams',
  'FirstName__c': 'Dartanion',
  'MiddleName__c': 'De Angelo',
  'BirthDate__c': datetime.date(1993, 5, 21)},
 {'EmployeeID__c': '000159108',
  'LastName__c': 'Rodney',
  'FirstName__c': 'Heather',
  'MiddleName__c': 'Nicole',
  'BirthDate__c': datetime.date(1994, 6, 7)}]

In [22]:
#Export data (dict) to SF
for rec in staff_data_records:

    record = {
        'EmployeeID__c':rec['EmployeeID__c'],
        'LastName__c':rec['LastName__c'],
        'FirstName__c':rec['FirstName__c'],
        'MiddleName__c':rec['MiddleName__c'], 
        'BirthDate__c' : str(rec['BirthDate__c'])
       }
    
    try:
        sf.staff__c.create(record)
    except Exception as e:
        print(e)

## Staff Assignment Table
### Create staff look up table

In [23]:
#step 1
staff_lookup_list = []

# The `Name` column in the primary key in Salesforce objects
# The Salesforce query language is called SOQL 
data = sf.query_all_iter("SELECT EmployeeID__c, Name FROM staff__c")
for row in data:
    rec = {
        'ID_Staff__c': row['Name'], # this is a critical line of code
        'EmployeeID__c': row['EmployeeID__c']
    }
    staff_lookup_list.append(rec)
    
staff_lookup_list

[{'ID_Staff__c': 'a024x000002hLz3', 'EmployeeID__c': '000130109'},
 {'ID_Staff__c': 'a024x000002hLyy', 'EmployeeID__c': '000184220'},
 {'ID_Staff__c': 'a024x000002hLz8', 'EmployeeID__c': '000160655'},
 {'ID_Staff__c': 'a024x000002hLzD', 'EmployeeID__c': '000159108'}]

In [24]:
sf_staff_lookup_df = pd.DataFrame(staff_lookup_list)
sf_staff_lookup_df

,ID_Staff__c,EmployeeID__c
0,a024x000002hLz3,000130109
1,a024x000002hLyy,000184220
2,a024x000002hLz8,000160655
3,a024x000002hLzD,000159108


### Create class look up table

In [25]:
#Step 2a
sf_class_lookup_list = []

# The `Name` column in the primary key in Salesforce objects
# The Salesforce query language is called SOQL 

query ='SELECT ID_Course__c, Name, Section__c FROM Class__c'

data = sf.query_all_iter(query)
for row in data:
    rec = {
        'ID_Class__c': row['Name'], # this is a critical line of code
        'ID_Course__c': row['ID_Course__c'],
        'Section__c': row['Section__c']
        
    }
    sf_class_lookup_list.append(rec)
    
for rec in sf_class_lookup_list:
    rec['ID_Course__c'] = rec['ID_Course__c'][:-3] 
    
sf_class_lookup_list

[{'ID_Class__c': 'a014x000008WO51',
  'ID_Course__c': 'a004x000003VR92',
  'Section__c': 'GWU-ARL-DATA-PT-09-0'},
 {'ID_Class__c': 'a014x000008WO5G',
  'ID_Course__c': 'a004x000003VR9C',
  'Section__c': 'GWARL201905UIUX3'},
 {'ID_Class__c': 'a014x000008WO56',
  'ID_Course__c': 'a004x000003VR92',
  'Section__c': 'GWDC201805DATA3'},
 {'ID_Class__c': 'a014x000008WO5B',
  'ID_Course__c': 'a004x000003VR97',
  'Section__c': 'GWARL201905WEB3'}]

In [27]:
sf_class_lookup_df = pd.DataFrame(sf_class_lookup_list)
sf_class_lookup_df

,ID_Class__c,ID_Course__c,Section__c
0,a014x000008WO51,a004x000003VR92,GWU-ARL-DATA-PT-09-0
1,a014x000008WO5G,a004x000003VR9C,GWARL201905UIUX3
2,a014x000008WO56,a004x000003VR92,GWDC201805DATA3
3,a014x000008WO5B,a004x000003VR97,GWARL201905WEB3


In [28]:
#step 2b
sf_course_lookup_list = []

# The `Name` column in the primary key in Salesforce objects
# The Salesforce query language is called SOQL 

query ='''
    SELECT
        Course_Code__c,
        Name
        FROM Course__c
    '''

data = sf.query_all_iter(query)
for row in data:
    rec = {
        'ID_Course__c': row['Name'], # this is a critical line of code
        'Course_Code__c': row['Course_Code__c']
       
        
    }
    sf_course_lookup_list.append(rec)
    
print(sf_course_lookup_list)

sf_course_lookup_df = pd.DataFrame(sf_course_lookup_list)
sf_course_lookup_df


[{'ID_Course__c': 'a004x000003VR92', 'Course_Code__c': 'BC-DATAVIZ'}, {'ID_Course__c': 'a004x000003VR97', 'Course_Code__c': 'BC-WEBDEV'}, {'ID_Course__c': 'a004x000003VR9W', 'Course_Code__c': 'CIS-430'}, {'ID_Course__c': 'a004x000003VR9C', 'Course_Code__c': 'BC-UIUX'}, {'ID_Course__c': 'a004x000003VR9H', 'Course_Code__c': 'CIS-349'}, {'ID_Course__c': 'a004x000003VR9M', 'Course_Code__c': 'CIS-405'}, {'ID_Course__c': 'a004x000003VR9R', 'Course_Code__c': 'CIS-438'}, {'ID_Course__c': 'a004x000003VR9b', 'Course_Code__c': 'CIS-435'}]


,ID_Course__c,Course_Code__c
0,a004x000003VR92,BC-DATAVIZ
1,a004x000003VR97,BC-WEBDEV
2,a004x000003VR9W,CIS-430
3,a004x000003VR9C,BC-UIUX
4,a004x000003VR9H,CIS-349
5,a004x000003VR9M,CIS-405
6,a004x000003VR9R,CIS-438
7,a004x000003VR9b,CIS-435


In [29]:
# Step 2 c
# Merge class and course Salesforce tables to create a natural key class lookup table (extended class lookup)

class_lookup_ext = pd.merge(sf_course_lookup_df, sf_class_lookup_df)
class_lookup_ext

,ID_Course__c,Course_Code__c,ID_Class__c,Section__c
0,a004x000003VR92,BC-DATAVIZ,a014x000008WO51,GWU-ARL-DATA-PT-09-0
1,a004x000003VR92,BC-DATAVIZ,a014x000008WO56,GWDC201805DATA3
2,a004x000003VR97,BC-WEBDEV,a014x000008WO5B,GWARL201905WEB3
3,a004x000003VR9C,BC-UIUX,a014x000008WO5G,GWARL201905UIUX3


In [30]:
#Step 3
# Query the `staffassignment` table, staff and class table from MySQL
# extend the class table query to the course table o establish natural key records.
query = '''
    SELECT 
        sf.EmployeeID As 'EmployeeID__c',
        sa.Role AS Role__c,
        sa.StartDate AS StartDate__c,
        sa.EndDate AS EndDate__c,
        cl.Section AS Section__c,
        co.CourseCode AS CourseCode__c       
    FROM 
        staff sf
        INNER JOIN staffassignment sa
        ON sf.ID_Staff = sa.ID_Staff
        INNER JOIN class cl
        ON sa.ID_Class = cl.ID_class
        INNER JOIN course co
        on cl.ID_Course = co.ID_course
'''

sql_staffassign_data_df = pd.read_sql(query, conn)
sql_staffassign_data_df.head() 

,EmployeeID__c,Role__c,StartDate__c,EndDate__c,Section__c,CourseCode__c
0,000184220,Teacher Assistant,2020-09-16,None,GWU-ARL-DATA-PT-09-0,BC-DATAVIZ
1,000159108,Teacher Assistant,2020-09-16,None,GWU-ARL-DATA-PT-09-0,BC-DATAVIZ
2,000160655,Instructor,2020-09-16,None,GWU-ARL-DATA-PT-09-0,BC-DATAVIZ


In [32]:
#Step 4
## Merge MySQL staffassignment table with student lookup from step 1
student_xover_df = pd.merge(sql_staffassign_data_df, sf_staff_lookup_df)
#class_data_df.drop(columns = ['ID_Class','ID_Course','CourseName','CreditHours','BootCampCourse','Course_Code__c'], inplace=True)

student_xover_df.head()

,EmployeeID__c,Role__c,StartDate__c,EndDate__c,Section__c,CourseCode__c,ID_Staff__c
0,000184220,Teacher Assistant,2020-09-16,None,GWU-ARL-DATA-PT-09-0,BC-DATAVIZ,a024x000002hLyy
1,000159108,Teacher Assistant,2020-09-16,None,GWU-ARL-DATA-PT-09-0,BC-DATAVIZ,a024x000002hLzD
2,000160655,Instructor,2020-09-16,None,GWU-ARL-DATA-PT-09-0,BC-DATAVIZ,a024x000002hLz8


In [33]:
#Step 5
## Merge student xover df with extended class lookup ()
staffassign_data_df_mg2 = pd.merge(class_lookup_ext, student_xover_df)
#class_data_df.drop(columns = ['ID_Class','ID_Course','CourseName','CreditHours','BootCampCourse','Course_Code__c'], inplace=True)

staffassign_data_df_mg2.head()

,ID_Course__c,Course_Code__c,ID_Class__c,Section__c,EmployeeID__c,Role__c,StartDate__c,EndDate__c,CourseCode__c,ID_Staff__c
0,a004x000003VR92,BC-DATAVIZ,a014x000008WO51,GWU-ARL-DATA-PT-09-0,000184220,Teacher Assistant,2020-09-16,None,BC-DATAVIZ,a024x000002hLyy
1,a004x000003VR92,BC-DATAVIZ,a014x000008WO51,GWU-ARL-DATA-PT-09-0,000159108,Teacher Assistant,2020-09-16,None,BC-DATAVIZ,a024x000002hLzD
2,a004x000003VR92,BC-DATAVIZ,a014x000008WO51,GWU-ARL-DATA-PT-09-0,000160655,Instructor,2020-09-16,None,BC-DATAVIZ,a024x000002hLz8


In [42]:
staffassignment_df = staffassign_data_df_mg2[['ID_Class__c', 'ID_Staff__c', 'StartDate__c','EndDate__c', 'Role__c']]
#staffassignment_df['StartDate__c'] = pd.to_datetime(staffassignment_df['StartDate__c']).dt.date
staffassignment_df.head()

,ID_Class__c,ID_Staff__c,StartDate__c,EndDate__c,Role__c
0,a014x000008WO51,a024x000002hLyy,2020-09-16,None,Teacher Assistant
1,a014x000008WO51,a024x000002hLzD,2020-09-16,None,Teacher Assistant
2,a014x000008WO51,a024x000002hLz8,2020-09-16,None,Instructor


In [43]:
#Insert Staffasignment Records into Salesforce
staffassignment_records =  staffassignment_df.to_dict('records')

for rec in staffassignment_records:

    record = {
        'ID_Class__c':rec['ID_Class__c'],
        'ID_Staff__c':rec['ID_Staff__c'],
        'Role__c':rec['Role__c'],
        'StartDate__c':str(rec['StartDate__c']),
        'EndDate__c' : rec['EndDate__c']
     
    }
    
    try:
        sf.staffassignment__c.create(record)
    except Exception as e:
        print(e)


## Insert Student Records into Salesforce

In [47]:
student_data_df = pd.read_sql("SELECT * FROM student", conn)
student_data_df.head()

,ID_Student,StudentID,LastName,FirstName,MiddleName,BirthDate,Gender
0,33,25004961,Dartling,Heather,Alice,None,F
1,34,25003514,Dartling,Lana,Cecille,None,F
2,35,25005833,Dartling,Jessica,Dorothy,None,F
3,36,25002589,Dartling,Kimberly,Genevieve,None,F
4,37,25007185,Dartling,Katherine,Lynnette,None,F


In [48]:
student_data_df.rename(columns={
    'StudentID':'StudentID__c',
    'LastName':'LastName__c',
    'FirstName':'FirstName__c',
    'MiddleName':'MiddleName__c',
    'BirthDate' : 'BirthDate__c',
    'Gender' : 'Gender__c'
}, inplace=True)
student_data_df.head()

,ID_Student,StudentID__c,LastName__c,FirstName__c,MiddleName__c,BirthDate__c,Gender__c
0,33,25004961,Dartling,Heather,Alice,None,F
1,34,25003514,Dartling,Lana,Cecille,None,F
2,35,25005833,Dartling,Jessica,Dorothy,None,F
3,36,25002589,Dartling,Kimberly,Genevieve,None,F
4,37,25007185,Dartling,Katherine,Lynnette,None,F


In [49]:
student_data_df.drop(columns = ['ID_Student'], inplace=True)
#student_data_df['BirthDate__c'] = student_data_df['BirthDate__c'].replace({None: "Null"})
#student_data_df['BirthDate__c'] = pd.to_datetime(student_data_df['BirthDate__c']).dt.date

student_data_df.head()

,StudentID__c,LastName__c,FirstName__c,MiddleName__c,BirthDate__c,Gender__c
0,25004961,Dartling,Heather,Alice,None,F
1,25003514,Dartling,Lana,Cecille,None,F
2,25005833,Dartling,Jessica,Dorothy,None,F
3,25002589,Dartling,Kimberly,Genevieve,None,F
4,25007185,Dartling,Katherine,Lynnette,None,F


In [50]:
student_data_records = student_data_df.to_dict('records')
student_data_records

[{'StudentID__c': '25004961',
  'LastName__c': 'Dartling',
  'FirstName__c': 'Heather',
  'MiddleName__c': 'Alice',
  'BirthDate__c': None,
  'Gender__c': 'F'},
 {'StudentID__c': '25003514',
  'LastName__c': 'Dartling',
  'FirstName__c': 'Lana',
  'MiddleName__c': 'Cecille',
  'BirthDate__c': None,
  'Gender__c': 'F'},
 {'StudentID__c': '25005833',
  'LastName__c': 'Dartling',
  'FirstName__c': 'Jessica',
  'MiddleName__c': 'Dorothy',
  'BirthDate__c': None,
  'Gender__c': 'F'},
 {'StudentID__c': '25002589',
  'LastName__c': 'Dartling',
  'FirstName__c': 'Kimberly',
  'MiddleName__c': 'Genevieve',
  'BirthDate__c': None,
  'Gender__c': 'F'},
 {'StudentID__c': '25007185',
  'LastName__c': 'Dartling',
  'FirstName__c': 'Katherine',
  'MiddleName__c': 'Lynnette',
  'BirthDate__c': None,
  'Gender__c': 'F'},
 {'StudentID__c': '25006014',
  'LastName__c': 'Dartling',
  'FirstName__c': 'Precious',
  'MiddleName__c': 'Mariette',
  'BirthDate__c': None,
  'Gender__c': 'F'},
 {'StudentID__c': '

In [54]:
for rec in student_data_records:

    record = {
        'StudentID__c':rec['StudentID__c'],
        'LastName__c':rec['LastName__c'],
        'FirstName__c':rec['FirstName__c'],
        'MiddleName__c':rec['MiddleName__c'],
        'BirthDate__c' : rec['BirthDate__c'],
        'Gender__c' : rec['Gender__c']
    }
    
    try:
        sf.student__c.create(record)
    except Exception as e:
        print(e)

### student lookup table

In [55]:
student_lookup_list = []
data = sf.query_all_iter("SELECT StudentID__c, Name FROM student__c")
for row in data:
    rec = {
        'ID_Student__c': row['Name'], # this is a critical line of code
        'StudentID__c': row['StudentID__c']
    }
    student_lookup_list.append(rec)
    
student_lookup_list

[{'ID_Student__c': 'a034x000002jkhd', 'StudentID__c': '25006947'},
 {'ID_Student__c': 'a034x000002jkiH', 'StudentID__c': '25005443'},
 {'ID_Student__c': 'a034x000002jkhT', 'StudentID__c': '25005602'},
 {'ID_Student__c': 'a034x000002jkgk', 'StudentID__c': '25005833'},
 {'ID_Student__c': 'a034x000002jkhn', 'StudentID__c': '25006348'},
 {'ID_Student__c': 'a034x000002jkiR', 'StudentID__c': '25006027'},
 {'ID_Student__c': 'a034x000002jkgf', 'StudentID__c': '25004961'},
 {'ID_Student__c': 'a034x000002jkek', 'StudentID__c': '25003514'},
 {'ID_Student__c': 'a034x000002jkgp', 'StudentID__c': '25002589'},
 {'ID_Student__c': 'a034x000002jkgu', 'StudentID__c': '25007185'},
 {'ID_Student__c': 'a034x000002jkfT', 'StudentID__c': '25007528'},
 {'ID_Student__c': 'a034x000002jkh4', 'StudentID__c': '25003778'},
 {'ID_Student__c': 'a034x000002jkh9', 'StudentID__c': '25003605'},
 {'ID_Student__c': 'a034x000002jkhE', 'StudentID__c': '25002056'},
 {'ID_Student__c': 'a034x000002jkfs', 'StudentID__c': '2500415

In [56]:
student_lookup_df = pd.DataFrame(student_lookup_list)
student_lookup_df.head()

,ID_Student__c,StudentID__c
0,a034x000002jkhd,25006947
1,a034x000002jkiH,25005443
2,a034x000002jkhT,25005602
3,a034x000002jkgk,25005833
4,a034x000002jkhn,25006348


## Extended Class Lookup Table

In [57]:
# From Step 2c above
class_lookup_ext

,ID_Course__c,Course_Code__c,ID_Class__c,Section__c
0,a004x000003VR92,BC-DATAVIZ,a014x000008WO51,GWU-ARL-DATA-PT-09-0
1,a004x000003VR92,BC-DATAVIZ,a014x000008WO56,GWDC201805DATA3
2,a004x000003VR97,BC-WEBDEV,a014x000008WO5B,GWARL201905WEB3
3,a004x000003VR9C,BC-UIUX,a014x000008WO5G,GWARL201905UIUX3


In [62]:
#Step 3
# Query the `classparticipant` table, student and class table from MySQL
# extend the class table query to the course table o establish natural key records.
query = '''
    SELECT 
        st.StudentID As 'StudentID__c',
        cp.StartDate AS StartDate__c,
        cp.EndDate AS EndDate__c,
        cl.Section AS Section__c,
        co.CourseCode AS CourseCode__c       
    FROM 
        student st
        INNER JOIN classparticipant cp
        ON st.ID_Student = cp.ID_Student
        INNER JOIN class cl
        ON cp.ID_Class = cl.ID_class
        INNER JOIN course co
        on cl.ID_Course = co.ID_course
'''

sql_classpart_data_df = pd.read_sql(query, conn)
sql_classpart_data_df.head() 

,StudentID__c,StartDate__c,EndDate__c,Section__c,CourseCode__c
0,25004961,2020-09-16,None,GWU-ARL-DATA-PT-09-0,BC-DATAVIZ
1,25003514,2020-09-16,None,GWU-ARL-DATA-PT-09-0,BC-DATAVIZ
2,25005833,2020-09-16,None,GWU-ARL-DATA-PT-09-0,BC-DATAVIZ
3,25007334,2020-09-16,None,GWU-ARL-DATA-PT-09-0,BC-DATAVIZ
4,25002589,2020-09-16,None,GWU-ARL-DATA-PT-09-0,BC-DATAVIZ


In [66]:
# Merge Mysql classparticipant data with 1) student lookup and 2) extended class look up
classparticipant_df_mg1 = pd.merge(sql_classpart_data_df,student_lookup_df)

print(len(classparticipant_df_mg1))
classparticipant_df_mg1.head()

28


,StudentID__c,StartDate__c,EndDate__c,Section__c,CourseCode__c,ID_Student__c
0,25004961,2020-09-16,None,GWU-ARL-DATA-PT-09-0,BC-DATAVIZ,a034x000002jkgf
1,25003514,2020-09-16,None,GWU-ARL-DATA-PT-09-0,BC-DATAVIZ,a034x000002jkek
2,25005833,2020-09-16,None,GWU-ARL-DATA-PT-09-0,BC-DATAVIZ,a034x000002jkgk
3,25007334,2020-09-16,None,GWU-ARL-DATA-PT-09-0,BC-DATAVIZ,a034x000002jkig
4,25002589,2020-09-16,None,GWU-ARL-DATA-PT-09-0,BC-DATAVIZ,a034x000002jkgp


In [67]:
classparticipant_df_mg2 = pd.merge(classparticipant_df_mg1,class_lookup_ext)

print(len(classparticipant_df_mg2))
classparticipant_df_mg2.head()

28


,StudentID__c,StartDate__c,EndDate__c,Section__c,CourseCode__c,ID_Student__c,ID_Course__c,Course_Code__c,ID_Class__c
0,25004961,2020-09-16,None,GWU-ARL-DATA-PT-09-0,BC-DATAVIZ,a034x000002jkgf,a004x000003VR92,BC-DATAVIZ,a014x000008WO51
1,25003514,2020-09-16,None,GWU-ARL-DATA-PT-09-0,BC-DATAVIZ,a034x000002jkek,a004x000003VR92,BC-DATAVIZ,a014x000008WO51
2,25005833,2020-09-16,None,GWU-ARL-DATA-PT-09-0,BC-DATAVIZ,a034x000002jkgk,a004x000003VR92,BC-DATAVIZ,a014x000008WO51
3,25007334,2020-09-16,None,GWU-ARL-DATA-PT-09-0,BC-DATAVIZ,a034x000002jkig,a004x000003VR92,BC-DATAVIZ,a014x000008WO51
4,25002589,2020-09-16,None,GWU-ARL-DATA-PT-09-0,BC-DATAVIZ,a034x000002jkgp,a004x000003VR92,BC-DATAVIZ,a014x000008WO51


In [68]:
classparticipant_df = classparticipant_df_mg2[['ID_Class__c', 'ID_Student__c', 'StartDate__c', 'EndDate__c']]
classparticipant_df 

,ID_Class__c,ID_Student__c,StartDate__c,EndDate__c
0,a014x000008WO51,a034x000002jkgf,2020-09-16,None
1,a014x000008WO51,a034x000002jkek,2020-09-16,None
2,a014x000008WO51,a034x000002jkgk,2020-09-16,None
3,a014x000008WO51,a034x000002jkig,2020-09-16,None
4,a014x000008WO51,a034x000002jkgp,2020-09-16,None
5,a014x000008WO51,a034x000002jkgu,2020-09-16,None
6,a014x000008WO51,a034x000002jkgz,2020-09-16,None
7,a014x000008WO51,a034x000002jkfT,2020-09-16,None
8,a014x000008WO51,a034x000002jkh4,2020-09-16,None
9,a014x000008WO51,a034x000002jkh9,2020-09-16,None


In [69]:
classparticipant_records = classparticipant_df.to_dict('records')
classparticipant_records

[{'ID_Class__c': 'a014x000008WO51',
  'ID_Student__c': 'a034x000002jkgf',
  'StartDate__c': datetime.date(2020, 9, 16),
  'EndDate__c': None},
 {'ID_Class__c': 'a014x000008WO51',
  'ID_Student__c': 'a034x000002jkek',
  'StartDate__c': datetime.date(2020, 9, 16),
  'EndDate__c': None},
 {'ID_Class__c': 'a014x000008WO51',
  'ID_Student__c': 'a034x000002jkgk',
  'StartDate__c': datetime.date(2020, 9, 16),
  'EndDate__c': None},
 {'ID_Class__c': 'a014x000008WO51',
  'ID_Student__c': 'a034x000002jkig',
  'StartDate__c': datetime.date(2020, 9, 16),
  'EndDate__c': None},
 {'ID_Class__c': 'a014x000008WO51',
  'ID_Student__c': 'a034x000002jkgp',
  'StartDate__c': datetime.date(2020, 9, 16),
  'EndDate__c': None},
 {'ID_Class__c': 'a014x000008WO51',
  'ID_Student__c': 'a034x000002jkgu',
  'StartDate__c': datetime.date(2020, 9, 16),
  'EndDate__c': None},
 {'ID_Class__c': 'a014x000008WO51',
  'ID_Student__c': 'a034x000002jkgz',
  'StartDate__c': datetime.date(2020, 9, 16),
  'EndDate__c': None},

In [71]:
for rec in classparticipant_records:

    record = {
        'ID_Class__c': rec['ID_Class__c'],
        'ID_Student__c': rec['ID_Student__c'],
        'StartDate__c': str(rec['StartDate__c']),
        'EndDate__c': rec['EndDate__c'],
    }
    
    try:
        sf.classparticipant__c.create(record)
    except Exception as e:
        print(e)

## Example of Deleting Records

Select the IDs of the records first and then process the results.

Ultimately, you want a list of IDs in the end.


In [52]:
#class_records = sf.query("SELECT Id FROM student__c")
#recs_to_delete = [{'Id': r['Id']} for r in class_records['records']]
#recs_to_delete

[{'Id': 'a034x000002jhwvAAA'},
 {'Id': 'a034x000002jhx0AAA'},
 {'Id': 'a034x000002jhx5AAA'},
 {'Id': 'a034x000002jhxAAAQ'},
 {'Id': 'a034x000002jhxFAAQ'},
 {'Id': 'a034x000002jhxKAAQ'},
 {'Id': 'a034x000002jhxPAAQ'},
 {'Id': 'a034x000002jhxUAAQ'},
 {'Id': 'a034x000002jhxZAAQ'},
 {'Id': 'a034x000002jhxeAAA'},
 {'Id': 'a034x000002jhxjAAA'},
 {'Id': 'a034x000002jhxoAAA'},
 {'Id': 'a034x000002jhxtAAA'},
 {'Id': 'a034x000002jhxyAAA'},
 {'Id': 'a034x000002jhy3AAA'},
 {'Id': 'a034x000002jhy8AAA'},
 {'Id': 'a034x000002jhyDAAQ'},
 {'Id': 'a034x000002jhyIAAQ'},
 {'Id': 'a034x000002jhyNAAQ'},
 {'Id': 'a034x000002jhySAAQ'},
 {'Id': 'a034x000002jhyXAAQ'},
 {'Id': 'a034x000002jhycAAA'},
 {'Id': 'a034x000002jhyhAAA'},
 {'Id': 'a034x000002jhymAAA'},
 {'Id': 'a034x000002jhyrAAA'},
 {'Id': 'a034x000002jhywAAA'},
 {'Id': 'a034x000002jhz1AAA'},
 {'Id': 'a034x000002jhz6AAA'},
 {'Id': 'a034x000002jhzBAAQ'},
 {'Id': 'a034x000002jhzGAAQ'}]

In [ ]:
#sf.bulk.Course__c.delete(recs_to_delete)

In [53]:
#for rec in recs_to_delete:
#    try:
#        sf.student__c.delete(rec['Id'])
#    except Exception as e:
#        print(e)